In [1]:
import pandas as pd
import torch
from torch.utils.data import Dataset, random_split
from transformers import GPT2Tokenizer, TrainingArguments, Trainer, GPT2LMHeadModel

from sklearn.model_selection import train_test_split
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(device)

In [2]:
tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2', bos_token='<|startoftext|>',
                                          eos_token='<|endoftext|>', pad_token='<|pad|>')
model = GPT2LMHeadModel.from_pretrained('distilgpt2')
model.resize_token_embeddings(len(tokenizer))
model.to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50259, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [3]:
data_path = "netflix_titles.csv"
df = pd.read_csv(data_path)
description = df['description']
listed_in =df["listed_in"]

In [4]:
import numpy as np
listed_in_arr = [ category.strip().lower().split(",") for category in listed_in]
dist_listed = []
temp =set()

listed_in_new = []
for category in listed_in_arr:
    temp_list = []
    for sub_cat in category:
        temp.add(sub_cat.strip())
        temp_list.append(sub_cat.strip())
    listed_in_new.append(temp_list)
dist_listed = list(temp)


print(len(dist_listed))
print(dist_listed)

42
['movies', 'stand-up comedy & talk shows', 'docuseries', 'spanish-language tv shows', 'crime tv shows', 'anime series', 'music & musicals', 'anime features', 'stand-up comedy', 'lgbtq movies', 'tv thrillers', 'tv comedies', 'sci-fi & fantasy', 'comedies', 'independent movies', 'teen tv shows', "kids' tv", 'classic movies', 'tv shows', 'documentaries', 'children & family movies', 'faith & spirituality', 'british tv shows', 'horror movies', 'tv sci-fi & fantasy', 'korean tv shows', 'international tv shows', 'tv mysteries', 'tv horror', 'cult movies', 'tv action & adventure', 'reality tv', 'tv dramas', 'sports movies', 'classic & cult tv', 'science & nature tv', 'romantic movies', 'dramas', 'thrillers', 'international movies', 'action & adventure', 'romantic tv shows']


In [5]:
print(len(listed_in_new))
cat_list_newArr  =[]
for cat_list in listed_in_new:
    temp_arr = []
    for cur_onehot in dist_listed:
        issame = False
        
        for sub_cat_list in cat_list:
            if cur_onehot==sub_cat_list:
                issame=True
                
        if issame:
            temp_arr.append(1)
        else:
            temp_arr.append(0)
    cat_list_newArr.append((temp_arr))

len(cat_list_newArr[2])

8807


42

In [6]:
num_of_flags = len(cat_list_newArr[0])
max_length = max([len(tokenizer.encode(title.strip())) for title in description])+num_of_flags
print("max_length : ",max_length)

max_length :  104


In [7]:
len(description)

8807

In [8]:
amount = 8000
train_description, test_description = description[:amount],description[amount:]
train_flags , test_flags = cat_list_newArr[:amount],cat_list_newArr[amount:]


In [9]:
class NetflixDataset(Dataset):
    def __init__(self, txt_list,flags, tokenizer, max_length):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        for txt in txt_list:
            encodings_dict = tokenizer('<|startoftext|>' + txt + '<|endoftext|>',
                                       max_length=max_length, padding="max_length")
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.input_ids.append(torch.tensor(flags))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))
            self.attn_masks.append(torch.tensor(flags))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

In [12]:
dataset = NetflixDataset(train_description,train_flags, tokenizer, max_length=max_length)
train_size = int(0.9 * len(dataset))
train_dataset, val_dataset = random_split(dataset, [train_size, len(dataset) - train_size])

In [ ]:
"""
for temp in train_dataset:
    #print(temp[0])
    #print((temp[1]))
    
    #print(len(temp[0]))
    #print(len(temp[1]))
    if len(temp[0])!=len(temp[1]) and len(temp[0])==62:
        print(temp[0])
        print(temp[1])
    
        break

    """

In [13]:
training_args = TrainingArguments(output_dir='./results', num_train_epochs=1, logging_steps=100, save_steps=500,
                                  per_device_train_batch_size=1, per_device_eval_batch_size=1,
                                  warmup_steps=10, weight_decay=0.05, logging_dir='./logs', report_to = 'none')

In [14]:
next(model.parameters()).is_cuda

True

In [15]:
test_str = "Hello "

generated = tokenizer("<|startoftext|> "+ test_str, return_tensors="pt").input_ids
print(generated.cuda())
print("generated : ",len(generated))
sample_outputs = model.generate(generated.cuda(),no_repeat_ngram_size = 1,num_beams=20, num_return_sequences=2)

print(sample_outputs)
print(tokenizer.decode(sample_outputs[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tensor([[50257, 18435,   220]], device='cuda:0')
generated :  1


c:\Users\tharh\anaconda3\envs\tharhtet\lib\site-packages\transformers\generation_utils.py:1202: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


tensor([[50257, 18435,   220,   170, 50258, 27332,   225,   116,   171,   227,
           235,   159,   232,   115,  2343,   223,    97, 12520,   229,   118],
        [50257, 18435,   220,   170, 50258, 27332,   225,   116,   171,   227,
           235,   159,   232,   115,  2343,   223,    97, 12520,   229,   103]],
       device='cuda:0')
 Hello � ㊷ ⁤ 🇺


In [16]:
Trainer(model=model,  args=training_args, train_dataset=train_dataset, 
        eval_dataset=val_dataset, data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                                                              'attention_mask': torch.stack([f[1] for f in data]),
                                                              'labels': torch.stack([f[0] for f in data])}).train()

c:\Users\tharh\anaconda3\envs\tharhtet\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 14400
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 14400


  0%|          | 0/14400 [00:00<?, ?it/s]

RuntimeError: CUDA out of memory. Tried to allocate 2.88 GiB (GPU 0; 8.00 GiB total capacity; 4.37 GiB already allocated; 1.92 GiB free; 4.46 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
results = []
for title in test_titles: 
    new_titles = {
        'seed': title.split()[0],
        'predictions': []
    }
    generated = tokenizer("<|startoftext|> "+ title.split()[0], return_tensors="pt").input_ids.cuda()
    sample_outputs = model.generate(generated,no_repeat_ngram_size = 1,num_beams=20, num_return_sequences=2)
    
    new_titles['predictions'] = sample_outputs
    results.append(new_titles)
    

In [ ]:
for new_title in results:
    print(f"seed: {new_title['seed']}")
    for i, pred in enumerate(new_title['predictions']):
        print(f"{i+1}: {tokenizer.decode(pred, skip_special_tokens=True)}")